# 存储方式基准

在本笔记本中，我们将比较以下存储格式：
- CSV：逗号分隔的标准平面文本文件格式。
- HDF5：分层数据格式，最初由国家超级计算应用中心开发。它是一种快速且可扩展的数字数据存储格式，可在使用 PyTables 库的 pandas 中使用。
- Parquet：Apache Hadoop 生态系统的一部分，一种二进制、列式存储格式，可提供高效的数据压缩和编码，由 Cloudera 和 Twitter 开发。它可以通过 pandas 的原作者 Wes McKinney 领导的 `pyarrow` 库用于 pandas。

此notebook使用可配置为包含数字和/或文本数据的测试 使用DataFrame 来比较上述库的性能。对于 HDF5 库，我们测试了固定格式和表格格式。表格格式允许查询并可以附加到文件中。


## 用法


要重新创建本书中使用的图表，您需要使用不同的“data_type”设置和“generate_test_data”参数运行此笔记本两次，直至“Store Result”部分，如下所示：
1. `data_type='Numeric`: `numerical_cols=2000`, `text_cols=0` (default)
2. `data_type='Mixed`: `numerical_cols=1000`, `text_cols=1000`

## 导入和设置

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import string

In [3]:
sns.set_style('whitegrid')

In [4]:
results = {}

## 生成测试数据

可以配置为包含数字或文本数据或两者的测试“DataFrame”。对于 HDF5 库，我们测试了固定格式和表格格式。

In [5]:
def generate_test_data(nrows=100000, numerical_cols=2000, text_cols=0, text_length=10):
    s = "".join([random.choice(string.ascii_letters)
                 for _ in range(text_length)])
    data = pd.concat([pd.DataFrame(np.random.random(size=(nrows, numerical_cols))),
                      pd.DataFrame(np.full(shape=(nrows, text_cols), fill_value=s))],
                     axis=1, ignore_index=True)
    data.columns = [str(i) for i in data.columns]
    return data

In [6]:
data_type = 'Numeric'

In [7]:
df = generate_test_data(numerical_cols=1000, text_cols=1000)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 2000 entries, 0 to 1999
dtypes: float64(1000), object(1000)
memory usage: 1.5+ GB


## Parquet

### 文件大小

In [8]:
parquet_file = Path('test.parquet')

In [9]:
df.to_parquet(parquet_file)
size = parquet_file.stat().st_size

### 读性能

In [10]:
%%timeit -o
df = pd.read_parquet(parquet_file)

4.86 s ± 134 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 4.86 s ± 134 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [11]:
read = _

In [12]:
parquet_file.unlink()

### 写性能

In [13]:
%%timeit -o
df.to_parquet(parquet_file)
parquet_file.unlink()

43.5 s ± 1.13 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 43.5 s ± 1.13 s per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [14]:
write = _

### 结果

In [15]:
results['Parquet'] = {'read': np.mean(read.all_runs), 'write': np.mean(write.all_runs), 'size': size}

## HDF5

In [16]:
test_store = Path('index.h5')

### Fixed Format

#### 大小

In [17]:
with pd.HDFStore(test_store) as store:
    store.put('file', df)
size = test_store.stat().st_size

#### 读性能

In [18]:
%%timeit -o
with pd.HDFStore(test_store) as store:
    store.get('file')

2min 7s ± 2.73 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 2min 7s ± 2.73 s per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [19]:
read = _

In [20]:
test_store.unlink()

#### 写性能

In [21]:
%%timeit -o
with pd.HDFStore(test_store) as store:
    store.put('file', df)
test_store.unlink()

1min 10s ± 1.47 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 1min 10s ± 1.47 s per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [22]:
write = _

#### 结果

In [23]:
results['HDF Fixed'] = {'read': np.mean(read.all_runs), 'write': np.mean(write.all_runs), 'size': size}

### 表格格式

#### 大小

In [24]:
with pd.HDFStore(test_store) as store:
    store.append('file', df, format='t')
size = test_store.stat().st_size    

#### 读性能

In [ ]:
%%timeit -o
with pd.HDFStore(test_store) as store:
    df = store.get('file')

In [ ]:
read = _

In [ ]:
test_store.unlink()

#### 写性能

请注意，表格格式的“写入”不适用于文本数据。

In [ ]:
%%timeit -o
with pd.HDFStore(test_store) as store:
    store.append('file', df, format='t')
test_store.unlink()    

In [ ]:
write = _

#### 结果

In [ ]:
results['HDF Table'] = {'read': np.mean(read.all_runs), 'write': np.mean(write.all_runs), 'size': size}

### 表格选择

#### 大小

In [ ]:
with pd.HDFStore(test_store) as store:
    store.append('file', df, format='t', data_columns=['company', 'form'])
size = test_store.stat().st_size 

#### 读性能

In [ ]:
company = 'APPLE INC'

In [ ]:
%%timeit
with pd.HDFStore(test_store) as store:
    s = store.get('file')

In [ ]:
read = _

In [ ]:
test_store.unlink()

#### 写性能

In [ ]:
%%timeit
with pd.HDFStore(test_store) as store:
    store.append('file', df, format='t', data_columns=['company', 'form'])
test_store.unlink() 

In [ ]:
write = _

#### 结果

In [ ]:
results['HDF Select'] = {'read': np.mean(read.all_runs), 'write': np.mean(write.all_runs), 'size': size}

## CSV

In [ ]:
test_csv = Path('test.csv')

### 大小

In [ ]:
df.to_csv(test_csv)
test_csv.stat().st_size

### 读性能

In [ ]:
%%timeit -o
df = pd.read_csv(test_csv)

In [ ]:
read = _

In [ ]:
test_csv.unlink()  

### 写性能

In [ ]:
%%timeit -o
df.to_csv(test_csv)
test_csv.unlink()

In [ ]:
write = _

### 结果

In [ ]:
results['CSV'] = {'read': np.mean(read.all_runs), 'write': np.mean(write.all_runs), 'size': size}

## 存储结果

In [ ]:
pd.DataFrame(results).assign(Data=data_type).to_csv(f'{data_type}.csv')

## 显示结果

请按照上述“使用”下的说明运行笔记本两次，以创建两个包含不同测试数据结果的“csv”文件。

In [ ]:
df = (pd.read_csv('Numeric.csv', index_col=0)
      .append(pd.read_csv('Mixed.csv', index_col=0))
      .rename(columns=str.capitalize))
df.index.name='Storage'
df = df.set_index('Data', append=True).unstack()
df.Size /= 1e9

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(16, 4))
for i, op in enumerate(['Read', 'Write', 'Size']):
    flag= op in ['Read', 'Write']
    df.loc[:, op].plot.barh(title=op, ax=axes[i], logx=flag)
    if flag:
        axes[i].set_xlabel('seconds (log scale)')
    else:
        axes[i].set_xlabel('GB')
fig.tight_layout()
fig.savefig('storage', dpi=300);